In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [737 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,848 kB]
Get:13 http://archive.ubuntu.com

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [ ]:
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [5]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType
from pyspark.sql.functions import month, year

# Next we need to create the list of struct fields
schema = [StructField("id", StringType(), True),StructField("date", StringType(), True), StructField("date_built", IntegerType(), True),StructField("price", IntegerType(), True),StructField("bedrooms", IntegerType(), True),StructField("bathrooms", IntegerType(), True),StructField("sqft_living", IntegerType(), True),StructField("sqft_lot", IntegerType(), True),StructField("floors", IntegerType(), True),StructField("waterfront", IntegerType(), True),StructField("view", IntegerType(), True)]

# Pass in our fields
final = StructType(fields=schema)

# Read our data with our new schema (enforcing a schema rather than letting in infer)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), schema=final, sep=",", header=True)

# Create a new DataFrame with the column Year
# Save the year for both date fields as a new column
df = df.withColumn('date_year', year(df['date']))

df.printSchema()



root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_year: integer (nullable = true)



In [6]:
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|     2022|


In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('sales')

In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""SELECT date_year, ROUND(MEAN(price),2)
FROM sales
WHERE bedrooms == 4
GROUP BY date_year
ORDER BY date_year""").show()


+---------+---------------------+
|date_year|round(mean(price), 2)|
+---------+---------------------+
|     2019|             300263.7|
|     2020|            298353.78|
|     2021|            301819.44|
|     2022|            296363.88|
+---------+---------------------+



In [9]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT date_built, ROUND(MEAN(price),2)
FROM sales
WHERE bedrooms == 3
AND bathrooms == 3
GROUP BY date_built
ORDER BY date_built""").show()


+----------+---------------------+
|date_built|round(mean(price), 2)|
+----------+---------------------+
|      2010|            292859.62|
|      2011|            291117.47|
|      2012|            293683.19|
|      2013|            295962.27|
|      2014|            290852.27|
|      2015|             288770.3|
|      2016|            290555.07|
|      2017|            292676.79|
+----------+---------------------+



In [10]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

spark.sql("""SELECT date_built, ROUND(MEAN(price),2)
FROM sales
WHERE bedrooms == 3
AND bathrooms == 3
AND floors == 2
AND sqft_living >=2000
GROUP BY date_built
ORDER BY date_built""").show()


+----------+---------------------+
|date_built|round(mean(price), 2)|
+----------+---------------------+
|      2010|            285010.22|
|      2011|            276553.81|
|      2012|            307539.97|
|      2013|            303676.79|
|      2014|            298264.72|
|      2015|            297609.97|
|      2016|             293965.1|
|      2017|            280317.58|
+----------+---------------------+



In [11]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT view, ROUND(MEAN(price),2)
FROM sales
WHERE price >=350000
GROUP BY view
ORDER BY view DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------------------+
|view|round(mean(price), 2)|
+----+---------------------+
| 100|            1026669.5|
|  99|           1061201.42|
|  98|           1053739.33|
|  97|           1129040.15|
|  96|           1017815.92|
|  95|            1054325.6|
|  94|            1033536.2|
|  93|           1026006.06|
|  92|            970402.55|
|  91|           1137372.73|
|  90|           1062654.16|
|  89|           1107839.15|
|  88|           1031719.35|
|  87|            1072285.2|
|  86|           1070444.25|
|  85|           1056336.74|
|  84|           1117233.13|
|  83|           1033965.93|
|  82|            1063498.0|
|  81|           1053472.79|
+----+---------------------+
only showing top 20 rows

--- 1.1794350147247314 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")


DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached("sales")

True

In [14]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""SELECT view, ROUND(MEAN(price),2)
FROM sales
WHERE price >=350000
GROUP BY view
ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------------------+
|view|round(mean(price), 2)|
+----+---------------------+
| 100|            1026669.5|
|  99|           1061201.42|
|  98|           1053739.33|
|  97|           1129040.15|
|  96|           1017815.92|
|  95|            1054325.6|
|  94|            1033536.2|
|  93|           1026006.06|
|  92|            970402.55|
|  91|           1137372.73|
|  90|           1062654.16|
|  89|           1107839.15|
|  88|           1031719.35|
|  87|            1072285.2|
|  86|           1070444.25|
|  85|           1056336.74|
|  84|           1117233.13|
|  83|           1033965.93|
|  82|            1063498.0|
|  81|           1053472.79|
+----+---------------------+
only showing top 20 rows

--- 0.5459849834442139 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("sales_partitioned")


In [16]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('sales_partitioned')

In [17]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_sales_p')

In [18]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""SELECT view, ROUND(MEAN(price),2)
FROM p_sales_p
WHERE price >=350000
GROUP BY view
ORDER BY view DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------------------+
|view|round(mean(price), 2)|
+----+---------------------+
| 100|            1026669.5|
|  99|           1061201.42|
|  98|           1053739.33|
|  97|           1129040.15|
|  96|           1017815.92|
|  95|            1054325.6|
|  94|            1033536.2|
|  93|           1026006.06|
|  92|            970402.55|
|  91|           1137372.73|
|  90|           1062654.16|
|  89|           1107839.15|
|  88|           1031719.35|
|  87|            1072285.2|
|  86|           1070444.25|
|  85|           1056336.74|
|  84|           1117233.13|
|  83|           1033965.93|
|  82|            1063498.0|
|  81|           1053472.79|
+----+---------------------+
only showing top 20 rows

--- 1.0562515258789062 seconds ---


In [19]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [20]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("sales")

False